[Pytorch](#Pytorch)

[PyTorch基础概念](#PyTorch基础概念)

[通用代码实现流程(实现一个深度学习的代码流程)](#手写数字识别代码实现)

# Pytorch

pytorch是基于Lua的Torch对其Tensor上所有模块进行重构并新增了AutoGrad系统的python版本

几乎所有所有的深度学习框架都是基于计算图的, 计算图分静态与动态, 

> 静态图

先定义再运行, 一次定义多次运行,

静态图一旦创建就不能修改,  无法使用if,while, for-loop等常用的python语句, 在构建图的时候必须把所有可能出现的情况都包含景区, 这导致了静态图过于庞大, 可能占用过高的显存.
>动态图

在运行的过程中被定义, 在运行的时候构建, 可以多次构建多次运行. 

>pytorch的优势

- 1. 简洁, 抽象层次为tensor->variable(autograd)->nn.Module, 分别代表高维数组(张量),自动求导(变量),神经网络(层/模块),且抽象层次间联系紧密,可同时进行修改和操作
- 2. 速度快
- 3. 好用,比tf容易上手, 比keras更灵活
- 4. 易于调试

# [PyTorch基础概念 ](https://github.com/chenyuntc/pytorch-book/blob/master/chapter2-快速入门/)
### Tensor
Tensor是Pytorch中重要的数据结构, 可认为是一个高维数组, Tensor和numpy的ndarrays类似, 但看有使用GPU加速

In [1]:
import torch as t
t.__version__

'1.0.0'

In [2]:
t.manual_seed(20190406)
t.cuda.manual_seed(20190406)
x = t.rand(5,3)
x

tensor([[0.1020, 0.4409, 0.7245],
        [0.6740, 0.4470, 0.3892],
        [0.3160, 0.2897, 0.0461],
        [0.6536, 0.7484, 0.7170],
        [0.0975, 0.4740, 0.1082]])

x.add_(x)和x.t_()会改变x, 但x.add(x)和x.t()会返回一个新的Tensor, 而x不变

In [3]:
x.add(x)

tensor([[0.2040, 0.8818, 1.4489],
        [1.3480, 0.8940, 0.7785],
        [0.6319, 0.5794, 0.0923],
        [1.3073, 1.4968, 1.4339],
        [0.1950, 0.9480, 0.2164]])

In [4]:
x

tensor([[0.1020, 0.4409, 0.7245],
        [0.6740, 0.4470, 0.3892],
        [0.3160, 0.2897, 0.0461],
        [0.6536, 0.7484, 0.7170],
        [0.0975, 0.4740, 0.1082]])

In [5]:
x.add_(x)

tensor([[0.2040, 0.8818, 1.4489],
        [1.3480, 0.8940, 0.7785],
        [0.6319, 0.5794, 0.0923],
        [1.3073, 1.4968, 1.4339],
        [0.1950, 0.9480, 0.2164]])

In [6]:
x

tensor([[0.2040, 0.8818, 1.4489],
        [1.3480, 0.8940, 0.7785],
        [0.6319, 0.5794, 0.0923],
        [1.3073, 1.4968, 1.4339],
        [0.1950, 0.9480, 0.2164]])

Tensor和numpy数组间共同的操作非常容易且快速, 对于Tensor不支持的操作, 可以先转为numpy数组处理, 之后再转回Tensor

In [7]:
y = x.numpy()
y

array([[0.20403636, 0.88179016, 1.4489224 ],
       [1.3479862 , 0.89399457, 0.7784997 ],
       [0.6319251 , 0.5794474 , 0.09229195],
       [1.3072973 , 1.4967729 , 1.4339076 ],
       [0.19501972, 0.94798684, 0.21635413]], dtype=float32)

In [8]:
z = t.from_numpy(y)
z

tensor([[0.2040, 0.8818, 1.4489],
        [1.3480, 0.8940, 0.7785],
        [0.6319, 0.5794, 0.0923],
        [1.3073, 1.4968, 1.4339],
        [0.1950, 0.9480, 0.2164]])

Tensor可通过.cuda方法转为GPU的Tensor, 从而享受GPU带来的加速运算

### Autograd
Tensor关于autograd的属性主要有
- data. 保存张量数值
- grad. 保存张量的梯度
- grad_fn 指向一个Function对象, BP时用来计算输入的梯度

要使Tensor使用autograd功能，只需要设置tensor.requries_grad=True.

In [9]:
x = t.ones(2, 2, requires_grad=True)
x
# 等价于
# x = t.ones(2,2)
# x.requires_grad = True

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [10]:
y = x.sum()
y

tensor(4., grad_fn=<SumBackward0>)

In [11]:
y.data, y.grad_fn, y.grad

(tensor(4.), <SumBackward0 at 0x7ff8314d8e10>, None)

In [12]:
y.backward()
x.grad

tensor([[1., 1.],
        [1., 1.]])

In [13]:
y.backward()
x.grad

tensor([[2., 2.],
        [2., 2.]])

In [14]:
# 以下划线结束的函数是inplace操作，会修改自身的值，就像add_
x.grad.data.zero_()

tensor([[0., 0.],
        [0., 0.]])

In [15]:
y.backward()
x.grad

tensor([[1., 1.],
        [1., 1.]])

# 手写数字识别代码实现
### 定义网络
需要继承nn.Module，实现它的forward方法，把网络中具有可学习参数的层放在构造函数__init__中。如果某一层(如ReLU)不具有可学习的参数，则既可以放在构造函数中，也可以不放，通常不放.

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
lenet = LeNet()
print(lenet)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只要在nn.Module的子类中定义了forward函数，backward函数就会自动被实现(利用autograd)。在forward 函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致。

网络的可学习参数通过net.parameters()返回，net.named_parameters可同时返回可学习的参数及名称。

In [17]:
params = list(lenet.parameters())
len(params)

10

In [18]:
for name, param in lenet.named_parameters():
    print(name,':', param.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [19]:
inp = t.rand(1, 1, 32, 32)
out = lenet(inp)
out.size()

torch.Size([1, 10])

In [20]:
lenet.zero_grad()
out.backward(t.ones(1,10))

### 损失函数

In [34]:
y = lenet(inp)
y_ = t.FloatTensor(range(0,10)).view(1,10)
y_ = t.arange(0,10).view(1,10).float()
criterion = nn.MSELoss()
loss = criterion(y, y_)
loss

tensor(29.0072, grad_fn=<MseLossBackward>)

In [35]:
lenet.zero_grad()
print(lenet.conv1.bias.grad)

loss.backward()
print(lenet.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0941, -0.0444,  0.0130,  0.0017, -0.0054, -0.0047])


### 优化器

In [43]:
import torch.optim as optim
optimizer = optim.SGD(lenet.parameters(), lr=0.01)

optimizer.zero_grad()
for i in range(10):
    y = lenet(inp)
    loss = criterion(y, y_)

    loss.backward()

    optimizer.step()
    print(loss.data)

tensor(3.0897)
tensor(125.8324)
tensor(15810.3477)
tensor(822.7627)
tensor(4841.0610)
tensor(12.3571)
tensor(11.2755)
tensor(10.2020)
tensor(9.1451)
tensor(8.1132)
